In [624]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re

In [625]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [626]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [627]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report

In [628]:
pip install delayed

In [629]:
df_train = pd.read_csv("train.txt",delimiter=';',names=['text','sentiment','Mood'])
df_val = pd.read_csv("val.txt",delimiter=';',names=['text','sentiment','Mood'])

In [630]:
df = pd.concat([df_train,df_val])
df.reset_index(inplace=True,drop=True)

In [631]:
print("Shape of the DataFrame:",df.shape)
df.sample(5)

Shape of the DataFrame: (18000, 3)


,text,sentiment,Mood
5377,i get this gut feeling or am i just being para...,fear,NaN
14389,i feel you see there is always the possibility...,anger,NaN
3880,i have this feeling of security about the char...,fear,NaN
14203,i came to china feeling a little frightened of...,fear,NaN
9285,i did not feel love from the men who abused,sadness,NaN


In [632]:
df['Mood'] = df['sentiment']

In [633]:
df

,text,sentiment,Mood
0,i didnt feel humiliated,sadness,sadness
1,i can go from feeling so hopeless to so damned...,sadness,sadness
2,im grabbing a minute to post i feel greedy wrong,anger,anger
3,i am ever feeling nostalgic about the fireplac...,love,love
4,i am feeling grouchy,anger,anger
...,...,...,...
17995,im having ssa examination tomorrow in the morn...,sadness,sadness
17996,i constantly worry about their fight against n...,joy,joy
17997,i feel its important to share this info for th...,joy,joy
17998,i truly feel that if you are passionate enough...,joy,joy


In [634]:
def custom_encoder(df):
    df.replace(to_replace ="surprise", value =1, inplace=True)
    df.replace(to_replace ="love", value =1, inplace=True)
    df.replace(to_replace ="joy", value =1, inplace=True)
    df.replace(to_replace ="fear", value =0, inplace=True)
    df.replace(to_replace ="anger", value =0, inplace=True)
    df.replace(to_replace ="sadness", value =0, inplace=True)

In [635]:
custom_encoder(df['sentiment'])

In [636]:
df.head()

,text,sentiment,Mood
0,i didnt feel humiliated,0,sadness
1,i can go from feeling so hopeless to so damned...,0,sadness
2,im grabbing a minute to post i feel greedy wrong,0,anger
3,i am ever feeling nostalgic about the fireplac...,1,love
4,i am feeling grouchy,0,anger


In [637]:
import pickle
train =df
file = "train.pkl"
fileobj = open(file, 'wb')
pickle.dump(train, fileobj)
fileobj.close()

In [638]:
file = "train.pkl"
fileobj = open(file, 'rb')
train = pickle.load(fileobj)
print(train)

                                                    text  sentiment     Mood
0                                i didnt feel humiliated          0  sadness
1      i can go from feeling so hopeless to so damned...          0  sadness
2       im grabbing a minute to post i feel greedy wrong          0    anger
3      i am ever feeling nostalgic about the fireplac...          1     love
4                                   i am feeling grouchy          0    anger
...                                                  ...        ...      ...
17995  im having ssa examination tomorrow in the morn...          0  sadness
17996  i constantly worry about their fight against n...          1      joy
17997  i feel its important to share this info for th...          1      joy
17998  i truly feel that if you are passionate enough...          1      joy
17999  i feel like i just wanna buy any cute make up ...          1      joy

[18000 rows x 3 columns]


In [639]:
#add stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [640]:
from sklearn.feature_extraction .text import TfidfVectorizer

In [641]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words("english")
all_stopwords.remove("not")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [642]:
corpus=[]

for i in range(0,18000):
  review = re.sub('[^a-zA-Z]' , ' ' , df['text'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [643]:
corpus

['didnt feel humili',
 'go feel hopeless damn hope around someon care awak',
 'im grab minut post feel greedi wrong',
 'ever feel nostalg fireplac know still properti',
 'feel grouchi',
 'ive feel littl burden late wasnt sure',
 'ive take milligram time recommend amount ive fallen asleep lot faster also feel like funni',
 'feel confus life teenag jade year old man',
 'petrona year feel petrona perform well made huge profit',
 'feel romant',
 'feel like make suffer see mean someth',
 'feel run divin experi expect type spiritu encount',
 'think easiest time year feel dissatisfi',
 'feel low energi thirsti',
 'immens sympathi gener point possibl proto writer tri find time write corner life sign agent let alon publish contract feel littl preciou',
 'not feel reassur anxieti side',
 'didnt realli feel embarrass',
 'feel pretti pathet time',
 'start feel sentiment doll child began collect vintag barbi doll sixti',
 'feel compromis skeptic valu everi unit work put',
 'feel irrit reject withou

In [644]:
selected_text = corpus

In [645]:
df['selected_text'] = selected_text

In [646]:
cols = df.columns.tolist()

In [647]:
cols

['text', 'sentiment', 'Mood', 'selected_text']

In [648]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=18000)

In [649]:
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:, -1].values

In [650]:
#saveing bag of words dict in prediction
import pickle
bow_path = './drive/MyDrive/Colab Notebooks/bow.pkl'
pickle.dump(cv, open(bow_path, "wb"))

In [651]:
#train
df_train = pd.read_csv("test.txt",delimiter=';',names=['text','sentiment','Mood'])

In [652]:
x_train = pd.concat([df_test])

In [653]:
x_train.head()

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,0,sadness
1,im updating my blog because i feel shitty,0,sadness
2,i never make her separate from me because i do...,0,sadness
3,i left with my bouquet of red and yellow tulip...,1,joy
4,i was feeling a little vain when i did this one,0,sadness


In [654]:
x_train['Mood'] = x_train['sentiment']
x_train

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,0,0
1,im updating my blog because i feel shitty,0,0
2,i never make her separate from me because i do...,0,0
3,i left with my bouquet of red and yellow tulip...,1,1
4,i was feeling a little vain when i did this one,0,0
...,...,...,...
1995,i just keep feeling like someone is being unki...,0,0
1996,im feeling a little cranky negative after this...,0,0
1997,i feel that i am useful to my people and that ...,1,1
1998,im feeling more comfortable with derby i feel ...,1,1


In [655]:
def custom_encoder(x_train):
    x_train.replace(to_replace ="surprise", value =1, inplace=True)
    x_train.replace(to_replace ="love", value =1, inplace=True)
    x_train.replace(to_replace ="joy", value =1, inplace=True)
    x_train.replace(to_replace ="fear", value =0, inplace=True)
    x_train.replace(to_replace ="anger", value =0, inplace=True)
    x_train.replace(to_replace ="sadness", value =0, inplace=True)

In [656]:
custom_encoder(x_train['sentiment'])

In [657]:
x_train

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,0,0
1,im updating my blog because i feel shitty,0,0
2,i never make her separate from me because i do...,0,0
3,i left with my bouquet of red and yellow tulip...,1,1
4,i was feeling a little vain when i did this one,0,0
...,...,...,...
1995,i just keep feeling like someone is being unki...,0,0
1996,im feeling a little cranky negative after this...,0,0
1997,i feel that i am useful to my people and that ...,1,1
1998,im feeling more comfortable with derby i feel ...,1,1


In [658]:
df_train = x_train
df_train

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,0,0
1,im updating my blog because i feel shitty,0,0
2,i never make her separate from me because i do...,0,0
3,i left with my bouquet of red and yellow tulip...,1,1
4,i was feeling a little vain when i did this one,0,0
...,...,...,...
1995,i just keep feeling like someone is being unki...,0,0
1996,im feeling a little cranky negative after this...,0,0
1997,i feel that i am useful to my people and that ...,1,1
1998,im feeling more comfortable with derby i feel ...,1,1


In [659]:
#test
df_test = pd.read_csv("test.txt",delimiter=';',names=['text','sentiment','Mood'])

In [660]:
x_test = pd.concat([df_test])


In [661]:
x_test.head()

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,sadness,NaN
1,im updating my blog because i feel shitty,sadness,NaN
2,i never make her separate from me because i do...,sadness,NaN
3,i left with my bouquet of red and yellow tulip...,joy,NaN
4,i was feeling a little vain when i did this one,sadness,NaN


In [662]:
x_test['Mood'] = x_test['sentiment']
x_test

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,sadness,sadness
1,im updating my blog because i feel shitty,sadness,sadness
2,i never make her separate from me because i do...,sadness,sadness
3,i left with my bouquet of red and yellow tulip...,joy,joy
4,i was feeling a little vain when i did this one,sadness,sadness
...,...,...,...
1995,i just keep feeling like someone is being unki...,anger,anger
1996,im feeling a little cranky negative after this...,anger,anger
1997,i feel that i am useful to my people and that ...,joy,joy
1998,im feeling more comfortable with derby i feel ...,joy,joy


In [663]:
def custom_encoder(x_test):
    x_test.replace(to_replace ="surprise", value =1, inplace=True)
    x_test.replace(to_replace ="love", value =1, inplace=True)
    x_test.replace(to_replace ="joy", value =1, inplace=True)
    x_test.replace(to_replace ="fear", value =0, inplace=True)
    x_test.replace(to_replace ="anger", value =0, inplace=True)
    x_test.replace(to_replace ="sadness", value =0, inplace=True)

In [664]:
custom_encoder(x_test['sentiment'])

In [665]:
x_test

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,0,sadness
1,im updating my blog because i feel shitty,0,sadness
2,i never make her separate from me because i do...,0,sadness
3,i left with my bouquet of red and yellow tulip...,1,joy
4,i was feeling a little vain when i did this one,0,sadness
...,...,...,...
1995,i just keep feeling like someone is being unki...,0,anger
1996,im feeling a little cranky negative after this...,0,anger
1997,i feel that i am useful to my people and that ...,1,joy
1998,im feeling more comfortable with derby i feel ...,1,joy


In [666]:
df_test = x_test
df_test

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,0,sadness
1,im updating my blog because i feel shitty,0,sadness
2,i never make her separate from me because i do...,0,sadness
3,i left with my bouquet of red and yellow tulip...,1,joy
4,i was feeling a little vain when i did this one,0,sadness
...,...,...,...
1995,i just keep feeling like someone is being unki...,0,anger
1996,im feeling a little cranky negative after this...,0,anger
1997,i feel that i am useful to my people and that ...,1,joy
1998,im feeling more comfortable with derby i feel ...,1,joy


In [667]:
#val
df_val = pd.read_csv("test.txt",delimiter=';',names=['text','sentiment','Mood'])

In [668]:
x_val = pd.concat([df_val])

In [669]:
x_val.head()

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,sadness,NaN
1,im updating my blog because i feel shitty,sadness,NaN
2,i never make her separate from me because i do...,sadness,NaN
3,i left with my bouquet of red and yellow tulip...,joy,NaN
4,i was feeling a little vain when i did this one,sadness,NaN


In [670]:
x_val['Mood'] = x_val['sentiment']
x_val

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,sadness,sadness
1,im updating my blog because i feel shitty,sadness,sadness
2,i never make her separate from me because i do...,sadness,sadness
3,i left with my bouquet of red and yellow tulip...,joy,joy
4,i was feeling a little vain when i did this one,sadness,sadness
...,...,...,...
1995,i just keep feeling like someone is being unki...,anger,anger
1996,im feeling a little cranky negative after this...,anger,anger
1997,i feel that i am useful to my people and that ...,joy,joy
1998,im feeling more comfortable with derby i feel ...,joy,joy


In [671]:
def custom_encoder(x_val):
    x_val.replace(to_replace ="surprise", value =1, inplace=True)
    x_val.replace(to_replace ="love", value =1, inplace=True)
    x_val.replace(to_replace ="joy", value =1, inplace=True)
    x_val.replace(to_replace ="fear", value =0, inplace=True)
    x_val.replace(to_replace ="anger", value =0, inplace=True)
    x_val.replace(to_replace ="sadness", value =0, inplace=True)

In [672]:
custom_encoder(x_val['sentiment'])

In [673]:
x_val

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,0,sadness
1,im updating my blog because i feel shitty,0,sadness
2,i never make her separate from me because i do...,0,sadness
3,i left with my bouquet of red and yellow tulip...,1,joy
4,i was feeling a little vain when i did this one,0,sadness
...,...,...,...
1995,i just keep feeling like someone is being unki...,0,anger
1996,im feeling a little cranky negative after this...,0,anger
1997,i feel that i am useful to my people and that ...,1,joy
1998,im feeling more comfortable with derby i feel ...,1,joy


In [674]:
df_val = x_val
df_val

,text,sentiment,Mood
0,im feeling rather rotten so im not very ambiti...,0,sadness
1,im updating my blog because i feel shitty,0,sadness
2,i never make her separate from me because i do...,0,sadness
3,i left with my bouquet of red and yellow tulip...,1,joy
4,i was feeling a little vain when i did this one,0,sadness
...,...,...,...
1995,i just keep feeling like someone is being unki...,0,anger
1996,im feeling a little cranky negative after this...,0,anger
1997,i feel that i am useful to my people and that ...,1,joy
1998,im feeling more comfortable with derby i feel ...,1,joy


In [675]:
df_train.to_csv("./drive/MyDrive/Colab Notebooks/train.csv", sep='\t', encoding='UTF-8', index=False)

In [676]:
df_test.to_csv("./drive/MyDrive/Colab Notebooks/test.csv", sep='\t', encoding='UTF-8', index=False)

In [677]:
df_val.to_csv("./drive/MyDrive/Colab Notebooks/val.csv", sep='\t', encoding='UTF-8', index=False)

In [678]:
df.head()

,text,sentiment,Mood,selected_text
0,i didnt feel humiliated,0,sadness,didnt feel humili
1,i can go from feeling so hopeless to so damned...,0,sadness,go feel hopeless damn hope around someon care ...
2,im grabbing a minute to post i feel greedy wrong,0,anger,im grab minut post feel greedi wrong
3,i am ever feeling nostalgic about the fireplac...,1,love,ever feel nostalg fireplac know still properti
4,i am feeling grouchy,0,anger,feel grouchi


In [679]:
df.to_csv("./drive/MyDrive/Colab Notebooks/train dump.csv", sep='\t', encoding='UTF-8', index=False)